In [32]:
import pandas as pd
import numpy as np
from pathlib import Path

PATH = Path("merged_libur_cuaca_ispu_ndvi.csv")

df = pd.read_csv(PATH, sep=";")

In [33]:
# parse tanggal
df["tanggal"] = pd.to_datetime(df["tanggal"], dayfirst=True)

# sort
df = df.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

# drop kolom
DROP_COLS = ["max", "parameter_pencemar_kritis", "time", "id", "stasiun"]
df = df.drop(columns=DROP_COLS, errors="ignore")

# label mapping
LABEL_MAP = {"BAIK": 0, "SEDANG": 1, "TIDAK SEHAT": 2}
df = df[df["kategori"].notna()].copy()
df["target"] = df["kategori"].map(LABEL_MAP).astype(int)


# REINDEX 
def reindex_daily(g):
    idx = pd.date_range(g["tanggal"].min(), g["tanggal"].max(), freq="D")
    g = g.set_index("tanggal").reindex(idx)
    g["lokasi_clean"] = g["lokasi_clean"].iloc[0]
    return g.reset_index().rename(columns={"index": "tanggal"})

df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)


# PM2.5 HANDLING
df["pm25_missing"] = df["pm_duakomalima"].isna().astype(int)

median_pm25 = (
    df[df["tanggal"] >= "2021-01-01"]
    .groupby("lokasi_clean")["pm_duakomalima"]
    .median()
)

df["pm_duakomalima"] = df["pm_duakomalima"].fillna(
    df["lokasi_clean"].map(median_pm25)
)


# buang baris tanpa target (penting untuk training)
df = df[df["target"].notna()].copy()


# TIME FEATURES 
df["month"] = df["tanggal"].dt.month
df["dayofyear"] = df["tanggal"].dt.dayofyear
df["dayofweek"] = df["tanggal"].dt.dayofweek

df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
df["month_cos"] = np.cos(2*np.pi*df["month"]/12)

df["doy_sin"] = np.sin(2*np.pi*df["dayofyear"]/365)
df["doy_cos"] = np.cos(2*np.pi*df["dayofyear"]/365)

df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)

df.head()


C:\Users\USER\AppData\Local\Temp\ipykernel_22400\3085690969.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)


,tanggal,periode_data,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,kategori,temperature_2m_max (°C),...,pm25_missing,month,dayofyear,dayofweek,month_sin,month_cos,doy_sin,doy_cos,dow_sin,dow_cos
0,2010-01-01,201001.0,60.0,73.0,4.0,73.0,27.0,14.0,SEDANG,29.4,...,1,1,1,4,0.5,0.866025,0.017213,0.999852,-0.433884,-0.900969
1,2010-01-02,201001.0,32.0,73.0,2.0,16.0,33.0,9.0,BAIK,30.8,...,1,1,2,5,0.5,0.866025,0.034422,0.999407,-0.974928,-0.222521
2,2010-01-03,201001.0,27.0,73.0,2.0,19.0,20.0,9.0,BAIK,30.4,...,1,1,3,6,0.5,0.866025,0.051620,0.998667,-0.781831,0.623490
3,2010-01-04,201001.0,22.0,73.0,2.0,16.0,15.0,6.0,BAIK,30.3,...,1,1,4,0,0.5,0.866025,0.068802,0.997630,0.000000,1.000000
4,2010-01-05,201001.0,25.0,73.0,2.0,17.0,15.0,8.0,BAIK,29.9,...,1,1,5,1,0.5,0.866025,0.085965,0.996298,0.781831,0.623490


In [34]:
# fitur lag dan rolling

POLLUTANT_COLS = [
    "pm_sepuluh",
    "pm_duakomalima",
    "ozon",
    "nitrogen_dioksida",
    "sulfur_dioksida",
    "karbon_monoksida",
]

WEATHER_COLS = [
    "temperature_2m_mean (°C)",
    "relative_humidity_2m_mean (%)",
    "wind_speed_10m_mean (km/h)",
    "precipitation_sum (mm)",
    "cloud_cover_mean (%)",
    "surface_pressure_mean (hPa)",
]

LAG_FEATURES = POLLUTANT_COLS + WEATHER_COLS

LAGS = [1, 2, 3]
ROLL_WINDOWS = [3, 7]
def create_temporal_features(g):
    g = g.sort_values("tanggal")
    for col in LAG_FEATURES:
        for lag in LAGS:
            g[f"{col}_lag_{lag}"] = g[col].shift(lag)


    for col in POLLUTANT_COLS:
        for w in ROLL_WINDOWS:
            g[f"{col}_roll_mean_{w}"] = g[col].shift(1).rolling(w).mean()
            g[f"{col}_roll_std_{w}"]  = g[col].shift(1).rolling(w).std()

    return g

df = df.groupby("lokasi_clean", group_keys=False).apply(create_temporal_features)
lag_cols = [c for c in df.columns if "lag_" in c or "roll_" in c]

for col in lag_cols:
    df[f"{col}_isnan"] = df[col].isna().astype(int)


df.head()


C:\Users\USER\AppData\Local\Temp\ipykernel_22400\759381649.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("lokasi_clean", group_keys=False).apply(create_temporal_features)
C:\Users\USER\AppData\Local\Temp\ipykernel_22400\759381649.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_isnan"] = df[col].isna().astype(int)
C:\Users\USER\AppData\Local\Temp\ipykernel_22400\759381649.py:43: PerformanceWarning: DataFrame is highly fragmented. 

,tanggal,periode_data,pm_sepuluh,pm_duakomalima,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida,kategori,temperature_2m_max (°C),...,nitrogen_dioksida_roll_mean_7_isnan,nitrogen_dioksida_roll_std_7_isnan,sulfur_dioksida_roll_mean_3_isnan,sulfur_dioksida_roll_std_3_isnan,sulfur_dioksida_roll_mean_7_isnan,sulfur_dioksida_roll_std_7_isnan,karbon_monoksida_roll_mean_3_isnan,karbon_monoksida_roll_std_3_isnan,karbon_monoksida_roll_mean_7_isnan,karbon_monoksida_roll_std_7_isnan
0,2010-01-01,201001.0,60.0,73.0,4.0,73.0,27.0,14.0,SEDANG,29.4,...,1,1,1,1,1,1,1,1,1,1
1,2010-01-02,201001.0,32.0,73.0,2.0,16.0,33.0,9.0,BAIK,30.8,...,1,1,1,1,1,1,1,1,1,1
2,2010-01-03,201001.0,27.0,73.0,2.0,19.0,20.0,9.0,BAIK,30.4,...,1,1,1,1,1,1,1,1,1,1
3,2010-01-04,201001.0,22.0,73.0,2.0,16.0,15.0,6.0,BAIK,30.3,...,1,1,0,0,1,1,0,0,1,1
4,2010-01-05,201001.0,25.0,73.0,2.0,17.0,15.0,8.0,BAIK,29.9,...,1,1,0,0,1,1,0,0,1,1


In [35]:

df_model = df.dropna(subset=["target"]).reset_index(drop=True)

H = 30

rows = []

for loc, g in df_model.groupby("lokasi_clean"):
    g = g.sort_values("tanggal").reset_index(drop=True)

    # precompute lag sekali saja
    for col in POLLUTANT_COLS:
        g[f"{col}_lag1"] = g[col].shift(0)
        g[f"{col}_lag2"] = g[col].shift(1)
        g[f"{col}_lag3"] = g[col].shift(2)

    for h in range(1, H + 1):
        g_future = g.shift(-h)

        temp = pd.DataFrame({
            "lokasi_clean": loc,
            "tanggal": g["tanggal"],
            "horizon": h,
            "month": g["tanggal"].dt.month,
            "dayofyear": g["tanggal"].dt.dayofyear,
            "dayofweek": g["tanggal"].dt.dayofweek,
        })

        # fitur lag
        for col in POLLUTANT_COLS:
            temp[f"{col}_lag1"] = g[f"{col}_lag1"]
            temp[f"{col}_lag2"] = g[f"{col}_lag2"]
            temp[f"{col}_lag3"] = g[f"{col}_lag3"]

        # target masa depan
        for col in POLLUTANT_COLS:
            temp[f"target_{col}"] = g_future[col]

        rows.append(temp)

df_forecast = pd.concat(rows, ignore_index=True).dropna().reset_index(drop=True)




from sklearn.preprocessing import LabelEncoder

le_loc = LabelEncoder()
df_model["lokasi_enc"] = le_loc.fit_transform(df_model["lokasi_clean"])

df_forecast["lokasi_enc"] = le_loc.transform(df_forecast["lokasi_clean"])



In [36]:
import lightgbm as lgb

pollutant_models = {}

FEATURES_F = [
    c for c in df_forecast.columns
    if c.startswith(tuple(POLLUTANT_COLS))
    or c in ["month", "dayofyear", "dayofweek", "lokasi_enc", "horizon"]
]

for col in POLLUTANT_COLS:

    y_col = f"target_{col}"

    Xf = df_forecast[FEATURES_F]
    yf = df_forecast[y_col]

    train_data = lgb.Dataset(Xf, label=yf)

    params = {
        "objective": "regression",
        "metric": "l2",
        "learning_rate": 0.05,
        "num_leaves": 64,
        "verbosity": -1,
        "seed": 42,
    }

    model = lgb.train(params, train_data, num_boost_round=500)
    pollutant_models[col] = model


In [37]:
df_forecast.shape

(455085, 31)

In [38]:
from sklearn.preprocessing import LabelEncoder

le_loc = LabelEncoder()
df_model["lokasi_enc"] = le_loc.fit_transform(df_model["lokasi_clean"])


In [39]:


ISPU_FEATURES = POLLUTANT_COLS + [
    "month", "dayofyear", "dayofweek",
    "lokasi_enc"
]

X_ispu = df_model[ISPU_FEATURES]
y_ispu = df_model["target"]

train_data = lgb.Dataset(X_ispu, label=y_ispu)

ISPU_PARAMS = {
    "objective": "multiclass",
    "num_class": 3,
    "metric": "None",
    "learning_rate": 0.05,
    "num_leaves": 63,
    "min_data_in_leaf": 40,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbosity": -1,
    "seed": 42,
    "class_weight": "balanced"
}


ispu_model = lgb.train(ISPU_PARAMS, train_data, num_boost_round=500)

print("✅ ISPU model trained")


✅ ISPU model trained


In [40]:
from sklearn.metrics import f1_score
import numpy as np


probs = ispu_model.predict(X_ispu)
best_t = [0.33, 0.33, 0.33]

for c in range(3):
    best_f1 = 0
    for t in np.linspace(0.2, 0.6, 25):
        pred = probs.argmax(1)
        pred[probs[:, c] > t] = c
        f1 = f1_score(y_ispu, pred, average="macro")
        if f1 > best_f1:
            best_f1 = f1
            best_t[c] = t

print("Best threshold:", best_t)


Best threshold: [0.2, 0.2, 0.2]


In [41]:
sub = pd.read_csv("sample_submission.csv")

sub["tanggal"] = pd.to_datetime(sub["id"].str.split("_").str[0])
sub["lokasi_clean"] = sub["id"].str.split("_").str[1]
sub = sub.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

sub["lokasi_enc"] = le_loc_f.transform(sub["lokasi_clean"])


In [42]:
forecast_rows = []

for loc in sub["lokasi_clean"].unique():

    # ===== riwayat 3 hari terakhir =====
    hist = (
        df_model[df_model["lokasi_clean"] == loc]
        .sort_values("tanggal")
        .iloc[-3:][POLLUTANT_COLS]
        .values.tolist()
    )

    future_rows = (
        sub[sub["lokasi_clean"] == loc]
        .sort_values("tanggal")
    )

    for _, row in future_rows.iterrows():

        tgl = row["tanggal"]

        # ===== fitur forecast =====
        feat = {
            "horizon": 1,  # autoregressive → selalu 1-step
            "month": tgl.month,
            "dayofyear": tgl.dayofyear,
            "dayofweek": tgl.dayofweek,
            "lokasi_enc": row["lokasi_enc"],
        }

        # ===== lag dari history rolling =====
        for i, col in enumerate(POLLUTANT_COLS):
            feat[f"{col}_lag1"] = hist[-1][i]
            feat[f"{col}_lag2"] = hist[-2][i]
            feat[f"{col}_lag3"] = hist[-3][i]

        Xf = pd.DataFrame([feat])

        # ===== prediksi semua polutan =====
        new_vals = []
        preds_pol = {}

        for i, col in enumerate(POLLUTANT_COLS):
            pred = pollutant_models[col].predict(Xf)[0]
            new_vals.append(pred)
            preds_pol[col] = pred

        # ===== update history (rolling window) =====
        hist.append(new_vals)
        hist.pop(0)

        # ===== simpan =====
        feat.update(preds_pol)
        feat["id"] = row["id"]

        forecast_rows.append(feat)

df_future = pd.DataFrame(forecast_rows)


In [43]:


sub = pd.read_csv("sample_submission.csv")

sub["tanggal"] = pd.to_datetime(sub["id"].str.split("_").str[0])
sub["lokasi_clean"] = sub["id"].str.split("_").str[1]

sub = sub.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

# encode lokasi (PASTI sama dengan df_model)
sub["lokasi_enc"] = le_loc.transform(sub["lokasi_clean"])

# tanggal terakhir di data training per lokasi
last_dates = (
    df_model.groupby("lokasi_clean")["tanggal"]
    .max()
    .to_dict()
)


In [44]:
results = []

for _, row in df_future.iterrows():

    feat_ispu = {
        col: row[col] for col in POLLUTANT_COLS
    }

    feat_ispu.update({
        "month": row["month"],
        "dayofyear": row["dayofyear"],
        "dayofweek": row["dayofweek"],
        "lokasi_enc": row["lokasi_enc"],   
    })

    Xi = pd.DataFrame([feat_ispu])

    prob = ispu_model.predict(Xi)
    pred = prob.argmax(1)

    for c, t in enumerate(best_t):
        pred[prob[:, c] > t] = c

    results.append(int(pred[0]))


In [46]:
INV_LABEL_MAP = {0: "BAIK", 1: "SEDANG", 2: "TIDAK SEHAT"}

sub_final = pd.read_csv("sample_submission.csv")

pred_df = df_future[["id"]].copy()
pred_df["kategori"] = [INV_LABEL_MAP[i] for i in results]


sub_final = sub_final.merge(pred_df, on="id", how="left")

sub_final.to_csv("submission_step4.csv", index=False)

print("✅ submission_step4.csv siap upload")


✅ submission_step4.csv siap upload


In [ ]:
import pandas as pd

df = pd.read_csv("submission_step4.csv", sep=";")  
df.to_csv("submission_step_4.csv", index=False)
